In [1]:
import numpy as np
import math
from scipy.stats import multivariate_normal
class igmm:
    def __init__(self, X, dim, sigma_ini, tau):
        self.X = X
        self.dim = dim
        self.pi = list()
        self.mu = list()
        self.C = list()
        self.sp = list()
        self.sigma_ini = sigma_ini
        self.tau = tau
        
    def posterior_prob(self, x, component):
        mvn = multivariate_normal(mean = np.array(self.mu[component][0]), cov = np.array(self.C[component]))
        pdf_value = mvn.pdf(x)
        return self.pi[component]*pdf_value
    
    def createnewornot(self, x):
        createnew = True
        for i in range(len(self.pi)):
            novelty_criterion = self.tau/(((2*math.pi)**(dim/2))*math.sqrt(np.linalg.det(np.array(self.C[i]))))
            #print(f"Is this not a vector of length 2: {self.mu[i]}\n Dimensions: {np.shape(self.mu[i])}")
            mvn = multivariate_normal(mean = self.mu[i][0], cov = np.array(self.C[i]))
            pdf_value = mvn.pdf(x)
            if(pdf_value >= novelty_criterion):
                createnew = False
                return createnew
        return createnew
    
    def update(self, x):
        createnew = self.createnewornot(x)
        if(createnew == True):
            self.mu.append(np.array(x))
            self.C.append((self.sigma_ini**2)*np.eye(dim))
            self.sp.append(1)
            
            total_sum = np.sum(self.sp)
            for j in range(len(self.sp)-1):
                self.pi[j] = self.sp[j]/total_sum
            self.pi.append(self.sp[len(self.sp)-1]/total_sum)
        else:
            for j in range(len(self.sp)):
                posterior_value = self.posterior_prob(x,j)
                self.sp[j] += posterior_value
                prev_mu = self.mu[j]
                self.mu[j] = self.mu[j] + (posterior_value/self.sp[j])*(np.array(x)-self.mu[j])
                self.C[j] = self.C[j] - np.matmul(self.mu[j]-prev_mu,(self.mu[j]-prev_mu).T) + (posterior_value/self.sp[j])*(np.matmul((np.array(x)-self.mu[j]),(np.array(x)-self.mu[j]).T)-self.C[j])
            total_sum = np.sum(self.sp)
            for j in range(len(self.pi)):
                self.pi[j] = self.sp[j]/total_sum
        return
    
    def fit(self):
        iter = 1
        for x in self.X:
            self.update(x)
            print(f"\n\n\033[1mFor next sample {iter}:\033[0m")
            print_pi = np.array(self.pi)
            print_mean = np.array(self.mu)
            print_cov = np.array(self.C)
            print_sp = np.array(self.sp)
            for j in range(len(self.mu)):
                print(f"\033[1mComponent #{j+1}:\033[0m\nPrior Probability: {print_pi[j]}\nMean: {print_mean[j][0]}\nContribution of dataset: {print_sp[j]}\nCovariance:\n {print_cov[j]}")
            iter+=1
        return

In [2]:
class gen_samples:
    def __init__(self, pi, mu, C):
        self.X = list()
        self.pi = pi
        self.mu = mu
        self.C = C
        return
    
    def generate_samples(self, n_samples):
        iter = 1
        while(iter <= n_samples):
            z_i = np.argmax(np.random.multinomial(1, self.pi))
            sample = np.random.multivariate_normal(self.mu[z_i], self.C[z_i], 1)
            self.X.append(np.array(sample))
            iter += 1
        return

In [3]:
pi_value = [1/4, 3/4]
pi_value = np.array(pi_value)
mu_value = [[-5,0,0], [10,0,5]]
mu_value = np.array(mu_value)
C_value = [[[1,0,0],[0,1,0],[0,0,1]],[[1,0,0],[0,1,0],[0,0,1]]]
C_value = np.array(C_value)
sample_generator = gen_samples(pi_value, mu_value, C_value)
sample_generator.generate_samples(1000)
X = sample_generator.X
X = np.array(X)

In [4]:
sigma_threshold = (np.max(X)-np.min(X))/10
tau = 0.3
dim = 3
incremental_model = igmm(X,dim,sigma_threshold,tau)
incremental_model.fit()



For next sample 1:
Component #1:
Prior Probability: 1.0
Mean: [7.95458239 1.37022642 5.77186832]
Contribution of dataset: 1
Covariance:
 [[4.75955105 0.         0.        ]
 [0.         4.75955105 0.        ]
 [0.         0.         4.75955105]]


For next sample 2:
Component #1:
Prior Probability: 1.0
Mean: [7.95867805 1.36627506 5.76738181]
Contribution of dataset: 1.0024769366783304
Covariance:
 [[4.76888833 0.02109726 0.02109726]
 [0.02109726 4.76888833 0.02109726]
 [0.02109726 0.02109726 4.76888833]]


For next sample 3:
Component #1:
Prior Probability: 0.500618468216278
Mean: [7.95867805 1.36627506 5.76738181]
Contribution of dataset: 1.0024769366783304
Covariance:
 [[4.76888833 0.02109726 0.02109726]
 [0.02109726 4.76888833 0.02109726]
 [0.02109726 0.02109726 4.76888833]]
Component #2:
Prior Probability: 0.4993815317837221
Mean: [-5.79674783  1.34011247 -0.62351229]
Contribution of dataset: 1.0
Covariance:
 [[4.75955105 0.         0.        ]
 [0.         4.75955105 0.        



For next sample 122:
Component #1:
Prior Probability: 0.14720275530925414
Mean: [8.0574354  1.31178824 5.72670081]
Contribution of dataset: 1.0607186415515153
Covariance:
 [[4.88669865 0.39959825 0.39959825]
 [0.39959825 4.88669865 0.39959825]
 [0.39959825 0.39959825 4.88669865]]
Component #2:
Prior Probability: 0.14270002635982398
Mean: [-5.7784399   1.32511754 -0.60946873]
Contribution of dataset: 1.0282727235082
Covariance:
 [[4.73660621 0.10792071 0.10792071]
 [0.10792071 4.73660621 0.10792071]
 [0.10792071 0.10792071 4.73660621]]
Component #3:
Prior Probability: 0.15066158215855607
Mean: [11.0972962   0.62927524  3.92398487]
Contribution of dataset: 1.0856423741898458
Covariance:
 [[4.69540132 0.31131393 0.31131393]
 [0.31131393 4.69540132 0.31131393]
 [0.31131393 0.31131393 4.69540132]]
Component #4:
Prior Probability: 0.14066900802467142
Mean: [-3.35016958 -0.21423603  1.33538648]
Contribution of dataset: 1.0136375422243764
Covariance:
 [[4.77063117 0.07511542 0.07511542]
 [0.



For next sample 190:
Component #1:
Prior Probability: 0.14768499348719874
Mean: [8.08718233 1.29434027 5.71573691]
Contribution of dataset: 1.0800624346902974
Covariance:
 [[4.9146685  0.50793155 0.50793155]
 [0.50793155 4.9146685  0.50793155]
 [0.50793155 0.50793155 4.9146685 ]]
Component #2:
Prior Probability: 0.1418815172389342
Mean: [-5.77111244  1.31486674 -0.60307484]
Contribution of dataset: 1.0376199594031164
Covariance:
 [[4.73745434 0.15046562 0.15046562]
 [0.15046562 4.73745434 0.15046562]
 [0.15046562 0.15046562 4.73745434]]
Component #3:
Prior Probability: 0.15214488642999915
Mean: [11.07272672  0.6180634   3.94623595]
Contribution of dataset: 1.1126789024607795
Covariance:
 [[4.68925202 0.41169177 0.41169177]
 [0.41169177 4.68925202 0.41169177]
 [0.41169177 0.41169177 4.68925202]]
Component #4:
Prior Probability: 0.13985646169225197
Mean: [-3.36169977 -0.2129452   1.32607104]
Contribution of dataset: 1.0228101512263479
Covariance:
 [[4.77248833 0.11908218 0.11908218]
 [



For next sample 271:
Component #1:
Prior Probability: 0.14865017486987328
Mean: [8.11985476 1.2734962  5.70032578]
Contribution of dataset: 1.1064710548466732
Covariance:
 [[4.93062289 0.62906342 0.62906342]
 [0.62906342 4.93062289 0.62906342]
 [0.62906342 0.62906342 4.93062289]]
Component #2:
Prior Probability: 0.14045816039738376
Mean: [-5.76561052  1.3048074  -0.59742749]
Contribution of dataset: 1.045494154532702
Covariance:
 [[4.7405314  0.18808983 0.18808983]
 [0.18808983 4.7405314  0.18808983]
 [0.18808983 0.18808983 4.7405314 ]]
Component #3:
Prior Probability: 0.15419353273980319
Mean: [11.04147818  0.60531893  3.96879832]
Contribution of dataset: 1.1477327959452168
Covariance:
 [[4.68098572 0.53407012 0.53407012]
 [0.53407012 4.68098572 0.53407012]
 [0.53407012 0.53407012 4.68098572]]
Component #4:
Prior Probability: 0.13848227890763554
Mean: [-3.37145862 -0.21215462  1.31888621]
Contribution of dataset: 1.0307867673525148
Covariance:
 [[4.7749653  0.15756896 0.15756896]
 [



For next sample 338:
Component #1:
Prior Probability: 0.1490726853968031
Mean: [8.14485361 1.25507779 5.69022577]
Contribution of dataset: 1.1261934322895883
Covariance:
 [[4.94906722 0.72283848 0.72283848]
 [0.72283848 4.94906722 0.72283848]
 [0.72283848 0.72283848 4.94906722]]
Component #2:
Prior Probability: 0.13950070020035843
Mean: [-5.75735509  1.29237194 -0.59441748]
Contribution of dataset: 1.0538803399647596
Covariance:
 [[4.74388512 0.22766932 0.22766932]
 [0.22766932 4.74388512 0.22766932]
 [0.22766932 0.22766932 4.74388512]]
Component #3:
Prior Probability: 0.15559195466990958
Mean: [11.01913808  0.59063828  3.98868557]
Contribution of dataset: 1.1754442941705354
Covariance:
 [[4.67539699 0.62624632 0.62624632]
 [0.62624632 4.67539699 0.62624632]
 [0.62624632 0.62624632 4.67539699]]
Component #4:
Prior Probability: 0.1376466003825084
Mean: [-3.38224869 -0.21359896  1.30883256]
Contribution of dataset: 1.0398732464981466
Covariance:
 [[4.77111534 0.19406611 0.19406611]
 [0

...



For next sample 725:
Component #1:
Prior Probability: 0.12097768867415803
Mean: [8.2688736  1.16668439 5.6480978 ]
Contribution of dataset: 1.2286339254986438
Covariance:
 [[5.01063858 1.13678247 1.13678247]
 [1.13678247 5.01063858 1.13678247]
 [1.13678247 1.13678247 5.01063858]]
Component #2:
Prior Probability: 0.10804194093562859
Mean: [-5.73814247  1.25135884 -0.57457782]
Contribution of dataset: 1.0972601267640958
Covariance:
 [[4.71534194 0.37767311 0.37767311]
 [0.37767311 4.71534194 0.37767311]
 [0.37767311 0.37767311 4.71534194]]
Component #3:
Prior Probability: 0.13071461777132562
Mean: [10.91748619  0.53464842  4.08463583]
Contribution of dataset: 1.3275209314421654
Covariance:
 [[4.57883595 0.9935433  0.9935433 ]
 [0.9935433  4.57883595 0.9935433 ]
 [0.9935433  0.9935433  4.57883595]]
Component #4:
Prior Probability: 0.10587705363615703
Mean: [-3.4308558  -0.21120599  1.27351857]
Contribution of dataset: 1.0752738083762816
Covariance:
 [[4.80416918 0.37780725 0.37780725]
 



For next sample 786:
Component #1:
Prior Probability: 0.12151087694731207
Mean: [8.28872046 1.15637915 5.63839773]
Contribution of dataset: 1.246150265852392
Covariance:
 [[5.01266459 1.1932608  1.1932608 ]
 [1.1932608  5.01266459 1.1932608 ]
 [1.1932608  1.1932608  5.01266459]]
Component #2:
Prior Probability: 0.1074819946455725
Mean: [-5.73495339  1.24510797 -0.57223011]
Contribution of dataset: 1.1022775867217407
Covariance:
 [[4.7137455  0.39582132 0.39582132]
 [0.39582132 4.7137455  0.39582132]
 [0.39582132 0.39582132 4.7137455 ]]
Component #3:
Prior Probability: 0.13218203044389257
Mean: [10.90121448  0.52889484  4.09927209]
Contribution of dataset: 1.3555878824740013
Covariance:
 [[4.55043511 1.03937463 1.03937463]
 [1.03937463 4.55043511 1.03937463]
 [1.03937463 1.03937463 4.55043511]]
Component #4:
Prior Probability: 0.10534351458006322
Mean: [-3.43662151 -0.21224308  1.26822201]
Contribution of dataset: 1.0803464841809374
Covariance:
 [[4.80321227 0.39763396 0.39763396]
 [0



For next sample 840:
Component #1:
Prior Probability: 0.1216985988130441
Mean: [8.30397709 1.14783883 5.63179063]
Contribution of dataset: 1.2585792560600926
Covariance:
 [[5.0168499 1.2351643 1.2351643]
 [1.2351643 5.0168499 1.2351643]
 [1.2351643 1.2351643 5.0168499]]
Component #2:
Prior Probability: 0.10730800041318365
Mean: [-5.73099697  1.23869851 -0.57030913]
Contribution of dataset: 1.1097549573006673
Covariance:
 [[4.70521339 0.41638277 0.41638277]
 [0.41638277 4.70521339 0.41638277]
 [0.41638277 0.41638277 4.70521339]]
Component #3:
Prior Probability: 0.13318311143022663
Mean: [10.88915922  0.52255718  4.11002336]
Contribution of dataset: 1.377349475988024
Covariance:
 [[4.52753105 1.07194397 1.07194397]
 [1.07194397 4.52753105 1.07194397]
 [1.07194397 1.07194397 4.52753105]]
Component #4:
Prior Probability: 0.10495259114374181
Mean: [-3.44418117 -0.21044219  1.26173276]
Contribution of dataset: 1.0853958498420413
Covariance:
 [[4.8133386  0.42825546 0.42825546]
 [0.42825546



For next sample 899:
Component #1:
Prior Probability: 0.12186011315373209
Mean: [8.31635345 1.13802082 5.62467066]
Contribution of dataset: 1.2716627981756554
Covariance:
 [[5.01685578 1.27407817 1.27407817]
 [1.27407817 5.01685578 1.27407817]
 [1.27407817 1.27407817 5.01685578]]
Component #2:
Prior Probability: 0.10717962452270213
Mean: [-5.72498613  1.23192245 -0.56499596]
Contribution of dataset: 1.1184655725373522
Covariance:
 [[4.69902883 0.44359964 0.44359964]
 [0.44359964 4.69902883 0.44359964]
 [0.44359964 0.44359964 4.69902883]]
Component #3:
Prior Probability: 0.1339829875750013
Mean: [10.87740651  0.51319546  4.12076303]
Contribution of dataset: 1.3981702172934682
Covariance:
 [[4.51724408 1.1131156  1.1131156 ]
 [1.1131156  4.51724408 1.1131156 ]
 [1.1131156  1.1131156  4.51724408]]
Component #4:
Prior Probability: 0.10483477619066627
Mean: [-3.45213978 -0.20809823  1.25504167]
Contribution of dataset: 1.09399606964552
Covariance:
 [[4.80367713 0.45306639 0.45306639]
 [0.

Component #1:
Prior Probability: 0.12203935705075541
Mean: [8.33352395 1.12721348 5.61881998]
Contribution of dataset: 1.2858865437640496
Covariance:
 [[5.02694307 1.32556594 1.32556594]
 [1.32556594 5.02694307 1.32556594]
 [1.32556594 1.32556594 5.02694307]]
Component #2:
Prior Probability: 0.10687517558513886
Mean: [-5.72162828  1.22426033 -0.56020303]
Contribution of dataset: 1.1261068024981
Covariance:
 [[4.69811522 0.47156137 0.47156137]
 [0.47156137 4.69811522 0.47156137]
 [0.47156137 0.47156137 4.69811522]]
Component #3:
Prior Probability: 0.1351488605028994
Mean: [10.8654981   0.50735321  4.13364453]
Contribution of dataset: 1.4240168526407955
Covariance:
 [[4.49506247 1.15272067 1.15272067]
 [1.15272067 4.49506247 1.15272067]
 [1.15272067 1.15272067 4.49506247]]
Component #4:
Prior Probability: 0.1045190652086065
Mean: [-3.46019063 -0.20811734  1.24861762]
Contribution of dataset: 1.1012812814365152
Covariance:
 [[4.80318655 0.48135603 0.48135603]
 [0.48135603 4.80318655 0.481

In [5]:
import numpy as np
import math
from scipy.stats import chi2

class deletespurious:
    def __init__(self, params, dim, confidence):
        self.params = params
        self.dim = dim
        self.confidence = confidence
    
    def delete_trivial(self):
        continue_update = True
        while(continue_update):
            continue_update = False
            num_components = len(self.params)
            delete_threshold = (math.pi/10)*(math.e ** (-1*(math.pi/10)*num_components))
            for k in range(num_components):
                if(self.params[k][0] <= delete_threshold):
                    del self.params[k]
                    continue_update = True
                    break
        return
    
    def inconfidenceornot(self, i, j):
        mahalanobis_distance = np.matmul(np.matmul(self.params[i][1]-self.params[j][1],np.linalg.inv(self.params[i][2])),(self.params[i][1]-self.params[j][1]).T)
        lower_critical_value = chi2.ppf((1-self.confidence)/2, dim)
        upper_critical_value = chi2.ppf(1-(1-self.confidence)/2, dim)
        if(lower_critical_value <= mahalanobis_distance <= upper_critical_value):
            return 1
        return 0
    
    def createLM(self):
        LM = []
        num_components = len(self.params)
        for i in range(num_components):
            ith_entries = list()
            for j in range(num_components):
                ith_entries.append(self.inconfidenceornot(i,j))
            LM.append(ith_entries)
        LM = np.array(LM)
        return LM
    
    def updateLM(self, LM, sum_LM, index_to_remove):
        updated_LM = np.delete(np.delete(LM, index_to_remove, axis=0), index_to_remove, axis=1)
        updated_sum_LM = np.delete(sum_LM, index_to_remove)
        del self.params[index_to_remove]
        return updated_LM, updated_sum_LM
    
    def deleteLMspurious(self):
        sorted_params = sorted(self.params, key=lambda x: x[0], reverse=True)
        self.params = sorted_params
        self.delete_trivial()
        LM = self.createLM()
        sum_LM = np.sum(LM, axis=1)
        continue_update = True
        while(continue_update):
            continue_update = False
            num_components = len(sum_LM)
            for i in range(num_components):
                for j in range(num_components):
                    if(LM[i][j]==1):
                        if(sum_LM[j]>=2):
                            continue_update = True
                            updated_LM, updated_sumLM = self.updateLM(LM, sum_LM, i)
                            LM = updated_LM
                            sum_LM = updated_sumLM
                        elif(sum_LM[j]==1 and self.params[i][0] <= self.params[j][0]):
                            continue_update = True
                            updated_LM, updated_sumLM = self.updateLM(LM, sum_LM, i)
                            LM = updated_LM
                            sum_LM = updated_sumLM
                        break
                if(continue_update):
                    break
        total_spsum = sum(x[3] for x in self.params)
        for i in range(len(self.params)):
            self.params[i][0] = self.params[i][3]/total_spsum
        return

In [6]:
M = len(incremental_model.pi)
params = []
for k in range(M):
    current_components = list()
    current_components.append(incremental_model.pi[k])
    current_components.append(incremental_model.mu[k])
    current_components.append(incremental_model.C[k])
    current_components.append(incremental_model.sp[k])
    params.append(current_components)
dim = 3
confidence = 0.95
remove_spurious = deletespurious(params, dim, confidence)
remove_spurious.deleteLMspurious()
for k in range(len(remove_spurious.params)):
    print(f"\033[1mComponent #{k+1}:\033[0m\nPrior Probability:{remove_spurious.params[k][0]}\nMean:{remove_spurious.params[k][1]}\nContribution:{remove_spurious.params[k][3]}\nCovariance\n{remove_spurious.params[k][2]}")

Component #1:
Prior Probability:0.5471288651076028
Mean:[[10.59625693 -0.42800303  6.8469111 ]]
Contribution:1.2398775563287756
Covariance
[[4.97876426 1.14003759 1.14003759]
 [1.14003759 4.97876426 1.14003759]
 [1.14003759 1.14003759 4.97876426]]
Component #2:
Prior Probability:0.45287113489239716
Mean:[[-6.58080518 -0.86855114  2.36803921]]
Contribution:1.026275146261557
Covariance
[[4.85966356 0.22196863 0.22196863]
 [0.22196863 4.85966356 0.22196863]
 [0.22196863 0.22196863 4.85966356]]
